In [ ]:
import threading
import time
import socket
from _thread import *

print_lock = threading.Lock()

class OracleBotThread(threading.Thread):
    def __init__(self, client_socket, client_address):
        threading.Thread.__init__(self)
        self.client_socket = client_socket
        self.client_address = client_address

    def run(self):
        self.greet_user()
        self.handle_conversation()
        self.goodbye_user()

    def greet_user(self):
        self.send_message("Hello! I'm the Oracle. How can I help you today?")
        time.sleep(1)

    def handle_conversation(self):
        try:
            while True:
                user_input = self.receive_message()

                if "weather" in user_input.lower():
                    self.predict_weather()
                elif "bike hires" in user_input.lower():
                    self.predict_bike_hires()
                elif "goodbye" in user_input.lower():
                    break
                else:
                    self.send_message("I'm sorry, I didn't understand. Ask about weather or bike hires.")

        except Exception as e:
            print(f"Error in handle_conversation: {e}")

    def goodbye_user(self):
        self.send_message("Thank you for consulting the Oracle. Have a great day!")
        time.sleep(1)
        self.client_socket.close()
        print_lock.release()

    def predict_weather(self):
        # Replace this with your actual machine learning code for weather prediction
        predicted_weather = "Tomorrow’s maximum temperature will be 5 degrees Celsius, the minimum temperature will be 0 degrees Celsius, the precipitation level will be 1 mm, and the overall weather will be cloudy."
        self.send_message(predicted_weather)

    def predict_bike_hires(self):
        # Replace this with your actual machine learning code for bike hire prediction
        predicted_bike_hires = "The number of bike hires will be 150 for tomorrow."
        self.send_message(predicted_bike_hires)

    def send_message(self, message):
        try:
            print("Sending message:", message)
            self.client_socket.send(message.encode('utf-8'))
        except Exception as e:
            print(f"Error in send_message: {e}")

    def receive_message(self):
        try:
            data = self.client_socket.recv(1024)
            user_input = data.decode('utf-8')
            print("Received message:", user_input)
            return user_input
        except Exception as e:
            print(f"Error in receive_message: {e}")
            return ""

def Main():
    host = "127.0.0.1"
    port = 12345

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((host, port))
    print("Socket bound to port", port)

    s.listen(5)
    print("Socket is listening")

    while True:
        c, addr = s.accept()    # Establish connection with the client
        print_lock.acquire()    # Lock acquired by the client
        print("Connected to:", addr[0], ":", addr[1])

        oracle_thread = OracleBotThread(c, addr)
        oracle_thread.start()

if __name__ == "__main__":
    Main()